In [1]:
import copy
import lsqfit
import h5py
import numpy             as np
import gvar              as gv
import matplotlib.pyplot as plt
import corrfitter        as cf
from scipy import linalg as la
from tqdm  import tqdm

In [2]:
from b2heavy.TwoPointFunctions.utils     import correlation_diagnostics
from b2heavy.TwoPointFunctions.types2pts import CorrelatorIO, plot_effective_coeffs
from b2heavy.TwoPointFunctions.fitter    import StagFitter

In [3]:
BINSIZE  = {
    'MediumCoarse':13,
    'Coarse-2':    16,
    'Coarse-1':    11,
    'Coarse-Phys': 19,
    'Fine-1':      16,
    'Fine-Phys':   16,
    'SuperFine':   22
}
def Tmax(y,errmax=0.3):
    for i,el in enumerate(y):
        if abs(gv.sdev(el)/gv.mean(el))>errmax:
            return i
    return len(y)

In [4]:
ENSEMBLE = 'Coarse-1'
MOMENTUM = '300'
DATA_DIR = '/Users/pietro/code/data_analysis/BtoD/Alex/'

cov_specs = dict(scale=True, shrink=True, cutsvd=1e-12)

In [5]:
iod = CorrelatorIO(ENSEMBLE,'Dst',MOMENTUM,PathToDataDir=DATA_DIR)
Dst = StagFitter(
    io       = iod,
    jkBin    = BINSIZE[ENSEMBLE],
    smearing = ['1S-1S','d-d','d-1S']
)

iob = CorrelatorIO(ENSEMBLE,'B','000',PathToDataDir=DATA_DIR)
B  = StagFitter(
    io       = iob,
    jkBin    = BINSIZE[ENSEMBLE],
    smearing = ['1S-1S']
)

# Priors

In [6]:
effm_d,effa_d = Dst.meff((12,28),**cov_specs)
effm_b,effa_b = B.meff(  (14,22),**cov_specs)


effm = {'Dst':effm_d, 'B':effm_b}
effa = {'Dst':effa_d, 'B':effa_b}

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in sqrt
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in log
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in arccosh
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in sqrt
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in log
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heav

In [7]:
NEXC = 3

In [8]:
pr = {}

for m in ['Dst','B']:
    pr[f'dE.{m}']   = ['0.5(5)' for n in range(NEXC)]; pr[f'dE.{m}'][0] = f'{effm[m].mean:.2f}(0.5)'
    pr[f'dE.{m}.o'] = ['0.5(5)' for n in range(NEXC)]

In [9]:
ampl = {
    '1S': '1(1)',
    'd' : '0.01(50)'
}

for mes in ['Dst','B']:
    for smr,pol in effa[mes]:
        s1,s2 = smr.split('-')
        if s1==s2:
            aux = effa[mes][smr,pol] # * 2 * effm

            z0 = f'{aux.mean:.6f}({"1.0" if s1=="1S" else "0.5"})'

            pr[f'Z.{mes}.{s1}.{pol}']    = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(NEXC)]
            pr[f'Z.{mes}.{s1}.{pol}'][0] = z0 
            pr[f'Z.{mes}.{s1}.{pol}.o']  = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(NEXC)]

In [10]:
nV = int((NEXC * (NEXC + 1)) / 2)
pr['Vnn'] = gv.gvar(nV * ['0.0(5)'])
pr['Voo'] = gv.gvar(nV * ['0.0(5)'])
pr['Vno'] = gv.gvar(NEXC * [NEXC * ['0.0(5)']])
pr['Von'] = gv.gvar(NEXC * [NEXC * ['0.0(5)']])

In [11]:
# logprior = gv.BufferDict()
# for k,p in pr.items():
#     logprior[f'log({k})'] = gv.log(gv.gvar(p))

prior = gv.BufferDict()
for k,p in pr.items():
    prior[k] = gv.gvar(p)

for k,p in prior.items():
    print(k,p)

dE.Dst [1.16(50) 0.50(50) 0.50(50)]
dE.Dst.o [0.50(50) 0.50(50) 0.50(50)]
dE.B [1.93(50) 0.50(50) 0.50(50)]
dE.B.o [0.50(50) 0.50(50) 0.50(50)]
Z.Dst.1S.Bot [0.5(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Bot.o [1.0(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Par [0.7(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Par.o [1.0(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.d.Bot [0.01(50) 0.01(50) 0.01(50)]
Z.Dst.d.Bot.o [0.01(50) 0.01(50) 0.01(50)]
Z.Dst.d.Par [0.01(50) 0.01(50) 0.01(50)]
Z.Dst.d.Par.o [0.01(50) 0.01(50) 0.01(50)]
Z.B.1S.Unpol [2.4(1.0) 1.0(1.0) 1.0(1.0)]
Z.B.1S.Unpol.o [1.0(1.0) 1.0(1.0) 1.0(1.0)]
Vnn [0 ± 0.50 0 ± 0.50 0 ± 0.50 0 ± 0.50 0 ± 0.50 0 ± 0.50]
Voo [0 ± 0.50 0 ± 0.50 0 ± 0.50 0 ± 0.50 0 ± 0.50 0 ± 0.50]
Vno [[0 ± 0.50 0 ± 0.50 0 ± 0.50]
 [0 ± 0.50 0 ± 0.50 0 ± 0.50]
 [0 ± 0.50 0 ± 0.50 0 ± 0.50]]
Von [[0 ± 0.50 0 ± 0.50 0 ± 0.50]
 [0 ± 0.50 0 ± 0.50 0 ± 0.50]
 [0 ± 0.50 0 ± 0.50 0 ± 0.50]]


In [60]:
x = np.array([[1   , 2   , 3   ], [4   , 5   , 6   ]])
y = np.array([[10  , 20  , 30  ], [40  , 50  , 60  ]])
w = np.array([[100 , 200 , 300 ], [400 , 500 , 600 ]])
z = np.array([[1000, 2000, 3000], [4000, 5000, 6000]])
zz = np.array([x,y,w,z])

np.hstack(zz).mean(axis=0).reshape((4,3))

array([[2.5e+00, 3.5e+00, 4.5e+00],
       [2.5e+01, 3.5e+01, 4.5e+01],
       [2.5e+02, 3.5e+02, 4.5e+02],
       [2.5e+03, 3.5e+03, 4.5e+03]])

# Model building

In [12]:
from b2heavy.FnalHISQMetadata import params
from b2heavy.TwoPointFunctions.utils import jkCorr, compute_covariance

mdata = params(ENSEMBLE)

data = h5py.File(f'/Users/pietro/code/data_analysis/BtoD/Alex/Ensembles/FnalHISQ/a{mdata["naSpc"]}/{mdata["hdf5File2"]}')
data = data['data']

In [13]:
def format(v):
    return gv.gvar(
        v.mean(axis=0),
        np.cov(v,rowvar=False) * (v.shape[0]-1)
    )

In [14]:
s_he = 'k' + mdata['kBStr']
s_li = 'k' + mdata['kDStr']
s_qq = 'm' + mdata['mlStr']

data3 = {}
data3all = {}
for k in data.keys():
    st = k.split('_')
    if len(st)==12:
        src,cur,snk,tt,kh,s1,s2,s3,s4,mq,kl,pmom = st

        B_to_Dst  = kh==s_he and mq==s_qq  and kl==s_li
        collinear = cur[-1]==snk[-1]
        par       = collinear and cur[-1]=='1'

        if B_to_Dst and collinear and pmom=='p300':
            pol = 'Par' if par else 'Bot'
            smr = '1S' if s2=='1S' else 'd'
            data3[f'{src}->{cur}->{snk}.{tt}.{smr}.{pol}'] = format(jkCorr(data[k][:],bsize=BINSIZE[ENSEMBLE]))

            print(k)

P5_A1_V1_T12_k0.08574_RW_1S_rot_rot_m0.00507_k0.12201_p300
P5_A1_V1_T12_k0.08574_RW_RW_rot_rot_m0.00507_k0.12201_p300
P5_A1_V1_T13_k0.08574_RW_1S_rot_rot_m0.00507_k0.12201_p300
P5_A1_V1_T13_k0.08574_RW_RW_rot_rot_m0.00507_k0.12201_p300
P5_A2_V2_T12_k0.08574_RW_1S_rot_rot_m0.00507_k0.12201_p300
P5_A2_V2_T12_k0.08574_RW_RW_rot_rot_m0.00507_k0.12201_p300
P5_A2_V2_T13_k0.08574_RW_1S_rot_rot_m0.00507_k0.12201_p300
P5_A2_V2_T13_k0.08574_RW_RW_rot_rot_m0.00507_k0.12201_p300
P5_A3_V3_T12_k0.08574_RW_1S_rot_rot_m0.00507_k0.12201_p300
P5_A3_V3_T12_k0.08574_RW_RW_rot_rot_m0.00507_k0.12201_p300
P5_A3_V3_T13_k0.08574_RW_1S_rot_rot_m0.00507_k0.12201_p300
P5_A3_V3_T13_k0.08574_RW_RW_rot_rot_m0.00507_k0.12201_p300


In [45]:
models = []
for k in data3:
    corr,tt,smr,pol = k.split('.')
    src,cur,snk     = corr.split('->')

    if corr.endswith('3'):
        continue
    
    if corr.endswith('2'):
        othertag = f'{src}->{cur[0]}3->{snk[0]}3.{tt}.{smr}.{pol}'

    fcn = cf.Corr3(
        datatag   = k,
        T         = int(tt[1:]),
        tmin      = 2,
        dEa       = (f'dE.B'  ,f'dE.B.o'),
        dEb       = (f'dE.Dst',f'dE.Dst.o'),
        # a         = (f'Z.B.1S.{pol}',f'Z.B.1S.{pol}.o'),
        a         = (f'Z.B.1S.Unpol',f'Z.B.1S.Unpol.o'),
        b         = (f'Z.Dst.{smr}.{pol}',f'Z.Dst.{smr}.{pol}.o'),
        sa        = (1,-1),
        sb        = (1,-1),
        # otherdata = othertag if pol=='Bot' else None,
        Vnn='Vnn', Vno='Vno', Von='Von', Voo='Voo',
    )

    models.append(fcn)

In [46]:
xdatad,ydatad = Dst.format(**cov_specs)


for smr,pol in ydatad:
    sm1,sm2 = smr.split('-')
    
    data3['Dst',smr,pol] = ydatad

    fcn = cf.Corr2(
        datatag = ('Dst',smr,pol),
        tp      = Dst.Nt,
        s       = (1.,-1.),
        tdata   = xdatad,
        tmin    = 4,
        tmax    = Tmax(ydatad[smr,pol]),
        dE      = (f'dE.Dst',f'dE.Dst.o'),
        a       = (f'Z.Dst.{sm1}.{pol}',f'Z.Dst.{sm1}.{pol}.o'),
        b       = (f'Z.Dst.{sm2}.{pol}',f'Z.Dst.{sm2}.{pol}.o')
    )

    models.append(fcn)

In [47]:
xdatab,ydatab = B.format(**cov_specs)

for smr,pol in ydatab:
    sm1,sm2 = smr.split('-')

    data3['B',smr,pol] = ydatab

    fcn = cf.Corr2(
        datatag = ('B',smr,pol),
        tp      = Dst.Nt,
        s       = (1.,-1.),
        tdata   = xdatad,
        tmin    = 4,
        tmax    = Tmax(ydatab[smr,pol]),
        dE      = (f'dE.B',f'dE.B.o'),
        a       = (f'Z.B.{sm1}.{pol}',f'Z.B.{sm1}.{pol}.o'),
        b       = (f'Z.B.{sm2}.{pol}',f'Z.B.{sm2}.{pol}.o')
    )

    models.append(fcn)

In [48]:
fitter = cf.CorrFitter(models=models)
cfit = fitter.lsqfit(data=data3,prior=prior)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed